<h1>SMS SPAM DETECTION MODEL</h1>



In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("spam.csv",encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

data = data.rename(columns = {'v1':'label','v2':'message'})


In [4]:
data.describe()

,label,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
data.groupby('label').describe()

message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4

In [6]:
data['lenght']=data['message'].apply(len)
data.head()


,label,message,lenght
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [7]:
data.describe()

,lenght
count,5572.000000
mean,80.118808
std,59.690841
min,2.000000
25%,36.000000
50%,61.000000
75%,121.000000
max,910.000000


<p>From this plot we can say that spam messages are much more long than ham messages on an average</p>


In [9]:
#Converting our labels to numeric labels
# ham = 0 and spam = 1
data['label_num']=data.label.map({'ham':0,'spam':1})
data.head()

,label,message,lenght,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",111,0
1,ham,Ok lar... Joking wif u oni...,29,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,1
3,ham,U dun say so early hor... U c already then say...,49,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,0


In [11]:
#loading our features and target to train
x = data.message
y = data.label_num


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=11)

In [13]:
vect = CountVectorizer()

X_train = vect.fit_transform(x_train)

X_test = vect.transform(x_test)





In [14]:

svc = SVC(kernel = 'linear')
mnb = MultinomialNB(alpha =0.2)
gnb  = GaussianNB()
lr = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=100,random_state=11)
abc = AdaBoostClassifier(n_estimators =100,random_state=11)

In [15]:

def training(clf,x_train,Y_train):
    clf.fit(x_train,Y_train)

def predict(clf,X_test):
    return clf.predict(X_test)

In [16]:

classifier={'SVM': svc , 'MultinomialNB': mnb,'GaussianNB': gnb,'logistic': lr,'RandomForest': rfc,'Adaboost': abc}

In [19]:

score = []
for n,c in classifier.items():
    training(c,X_train.toarray(),y_train)
    pred = predict(c,X_test.toarray())
    score.append((n,[accuracy_score(y_test,pred,normalize=True)]))
    

In [26]:
score_df=pd.DataFrame.from_items(score,orient='index',columns=['scores'])

score_df['Accuracy (%)']=score_df['scores']*100
score_df

,scores,Accuracy (%)
SVC,0.989232,98.923187
MultinomialNB,0.988514,98.851400
GaussianNB,0.898062,89.806174
logistic,0.987078,98.707825
RandomForest,0.985642,98.564250
Adaboost,0.987796,98.779612


<p>We can see svm has the best accuracy of 0.989 followed by multinomial naiye bayes which have accuracy of 0.988</p>

<p>Also notice that Gaussian NB work very poorly</p>